In [1]:
!pip install -q -U peft bitsandbytes

In [2]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
peft_model_id = "Tirendaz/mistral-7b-dolly-fine-tuned"

config = PeftConfig.from_pretrained(peft_model_id)

In [4]:
from transformers import BitsAndBytesConfig

model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    return_dict=True,
    load_in_4bit=True,
    device_map='auto'
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path,
                                          padding_side = "right",
                                          add_eos_token = True)
tokenizer.pad_token = tokenizer.eos_token

In [6]:
fine_tuned_model = PeftModel.from_pretrained(model, peft_model_id)

In [7]:
from transformers import pipeline, logging

logging.set_verbosity(logging.CRITICAL)

pipe = pipeline(
    task="text-generation", 
    model=fine_tuned_model, 
    tokenizer=tokenizer, 
    eos_token_id=model.config.eos_token_id,
    max_new_tokens=100)

In [8]:
prompt = """
What is a Python?  Here is some context: Python is a high-level, general-purpose programming language.
"""
pipe = pipeline(task="text-generation", 
                model=fine_tuned_model, 
                tokenizer=tokenizer, 
                eos_token_id=model.config.eos_token_id, 
                max_new_tokens=100)

result = pipe(f"<s>[INST] {prompt} [/INST]")
generated = result[0]['generated_text']
print(generated[generated.find('[/INST]')+8:])

/opt/conda/lib/python3.10/site-packages/bitsandbytes/nn/modules.py:226: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_type=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(f'Input type into Linear4bit is torch.float16, but bnb_4bit_compute_type=torch.float32 (default). This will lead to slow inference or training speed.')


Python is a high-level, general-purpose programming language. Its design philosophy emphasizes code readability with the use of significant whitespace. Its syntax allows programmers to express concepts in fewer lines of code than would be possible in languages such as C++ or Java. The language provides constructs intended to enable clear programming on both a small and large scale.

Python is dynamically typed and garbage-collected. It has a large and comprehensive standard library. Used in a


In [9]:
prompt = """
Please summarize what Linkedin does. Here is some context: LinkedIn is a business and employment-focused social media platform
"""
pipe = pipeline(task="text-generation", 
                model=fine_tuned_model, 
                tokenizer=tokenizer, 
                eos_token_id=model.config.eos_token_id, 
                max_new_tokens=100)

result = pipe(f"<s>[INST] {prompt} [/INST]")
generated = result[0]['generated_text']
print(generated[generated.find('[/INST]')+8:])

LinkedIn is a business and employment-focused social media platform. It is used for professional networking, including employers posting jobs and job seekers posting their resumes. LinkedIn was founded in 2002 and is headquartered in Sunnyvale, California. It is owned by Microsoft.

LinkedIn is a business and employment-focused social media platform. It is used for professional networking, including employers posting jobs and job seekers posting their resumes.
